In [12]:
import numpy as np
from scipy import io as sio
from matplotlib import pyplot as plt
import inspect

In [16]:
import typhon as tp
import typhon.retrieval.bmci as tprb

In [23]:
# Define a function to easily see the source code of a function. 
def showfunc(functiontoshow):
    print(inspect.getsource(functiontoshow))

In [34]:
showfunc(tp.oem.error_covariance_matrix)

def error_covariance_matrix(K, S_a, S_y):
    """Calculate the error covariance matrix.

    Parameters:
        K (np.array): Simulated Jacobians.
        S_a (np.array): A priori error covariance matrix.
        S_y (np.array): Measurement covariance matrix.

    Returns:
        np.array: Measurement error covariance matrix.
    """
    return inv(K.T @ inv(S_y) @ K + inv(S_a))



In [36]:
J = tp.arts.xml.load("./ClearSky_1D_jacobian.xml")
J.shape

(22, 50)

In [ ]:
tp.oem.

In [18]:
Tb_simulated = tp.arts.xml.load("./ClearSky_1D_Tb.xml")
Tb_BosungObs = np.array([26.193, 23.754, 22.435, 18.510, 15.617, 12.566, 12.388, 11.472, 
                         107.09, 123.677, 147.536, 178.933, 216.787, 250.225, 273.193, 282.353, 
                         286.576, 288.430, 289.69, 290.449, 291.361, 291.436])